In [ ]:
from loaddata import *
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
train_data1=readtraining_data("inter_a_b/train_XX.p","inter_a_b/train_YY.npy","inter_a_b/train_pos.npy")
train_data2=readtraining_data("inter_a_b/sur_XX.p","inter_a_b/sur_YY.npy","inter_a_b/sur_pos.npy")
train_data=mergedata(train_data1,train_data2)
train_data.print_info()
output_class = 4
print(train_data.labels.shape)
#create training data
stats,min_frac=count_stats(output_class,train_data.labels,train_data.elements)
min_frac=0.99*min_frac
train_data_1=select_data(train_data,min_frac,stats)
print("====New Data====")
train_data=train_data_1
train_data.print_info()
num_training = train_data.elements
count_stats(output_class,train_data.labels,train_data.elements)
print("max value:",np.max(train_data.feature_vec))

In [ ]:
ii=np.argwhere(train_data.labels==2)
viz_val=ii[15][0]
fig, (ax0, ax1,ax2,ax3) = plt.subplots(ncols=4,figsize=(16, 8))
ax0.imshow(train_data.feature_vec[viz_val,:,:,0],cmap=plt.cm.cool)
ax1.imshow(train_data.feature_vec[viz_val,:,:,1],cmap=plt.cm.cool)
ax2.imshow(train_data.feature_vec[viz_val,:,:,2],cmap=plt.cm.cool)
tt=(train_data.feature_vec[viz_val,:,:,0]/2.0)+(train_data.feature_vec[viz_val,:,:,1]*1.0)\
+(train_data.feature_vec[viz_val,:,:,2]/2.0)
ax3.imshow(tt,cmap=plt.cm.cool)    
plt.show()

In [ ]:
#defination of all the kernels
def conv2d(input, kernel_size, stride, num_filter, name = 'conv2d'):
    with tf.variable_scope(name):
        stride_shape = [1, stride, stride, 1]
        filter_shape = [kernel_size, kernel_size, input.get_shape()[3], num_filter]

        W = tf.get_variable('w', filter_shape, tf.float32, tf.random_normal_initializer(0.0, 0.02))
        b = tf.get_variable('b', [1, 1, 1, num_filter], initializer = tf.constant_initializer(0.0))
        return tf.nn.conv2d(input, W, stride_shape, padding = 'SAME') + b
    
def conv2d_transpose(input, kernel_size, stride, num_filter, name = 'conv2d_transpose'):
    with tf.variable_scope(name):
        stride_shape = [1, stride, stride, 1]
        filter_shape = [kernel_size, kernel_size, num_filter, input.get_shape()[3]]
        output_shape = tf.stack([tf.shape(input)[0], tf.shape(input)[1] * 2, tf.shape(input)[2] * 2, num_filter])

        W = tf.get_variable('w', filter_shape, tf.float32, tf.random_normal_initializer(0.0, 0.02))
        b = tf.get_variable('b', [1, 1, 1, num_filter], initializer = tf.constant_initializer(0.0))
        return tf.nn.conv2d_transpose(input, W, output_shape, stride_shape, padding = 'SAME') + b
    
def fc(input, num_output, name = 'fc'):
    with tf.variable_scope(name):
        num_input = input.get_shape()[1]
        W = tf.get_variable('w', [num_input, num_output], tf.float32, tf.random_normal_initializer(0.0, 0.02))
        b = tf.get_variable('b', [num_output], initializer = tf.constant_initializer(0.0))
        return tf.matmul(input, W) + b

def max_pool(input, kernel_size, stride):
    ksize = [1, kernel_size, kernel_size, 1]
    strides = [1, stride, stride, 1]
    return tf.nn.max_pool(input, ksize=ksize, strides=strides, padding='SAME')
    
def dropout(input,keep_prob,is_train):
    return tf.layers.dropout(input,keep_prob,training=is_train)

def batch_norm(input, is_training):
    out = tf.contrib.layers.batch_norm(input, decay = 0.99, center = True, scale = True,
                                       is_training = is_training, updates_collections = None)
    return out

def leaky_relu(input, alpha = 0.2):
    return tf.maximum(alpha * input, input)

In [ ]:
class VAE(object):
    def __init__(self):
        self.num_epoch = 150
        self.input_w = 64
        self.batch_size = 64
        self.log_step = 50
        self.learning_rate = 1e-3
        self.n_latent = 20
        self.is_train  = tf.placeholder(tf.bool)
        self.keep_prob = 0.50
        self.X = tf.placeholder(tf.float32, [None, self.input_w, self.input_w, 3])
        self.X_flat = tf.reshape(self.X, shape=[-1, self.input_w * self.input_w * 3])
        print(str(self.X.get_shape()))
        self._build_model()
        #self.sess = tf.InteractiveSession()
        #self.sess.run(tf.global_variables_initializer())
        
    def encoder(self,input):
         with tf.variable_scope("encoder", reuse=None):
                en_conv1 =  conv2d(input, 4, 2, 32, 'conv1')
                en_relu1 = leaky_relu(en_conv1)
                en_drop1 = dropout(en_relu1,self.keep_prob,self.is_train)
                print('conv1 layer: ' + str(en_relu1.get_shape()))
                en_conv2 =  conv2d(en_drop1, 4, 2, 64, 'conv2')
                en_relu2 = leaky_relu(en_conv2)
                en_drop2 = dropout(en_relu2,self.keep_prob,self.is_train)
                print('conv2 layer: ' + str(en_relu2.get_shape()))
                en_conv3 =  conv2d(en_drop2, 4, 2, 128, 'conv3')
                en_relu3 = leaky_relu(en_conv3)
                en_drop3 = dropout(en_relu3,self.keep_prob,self.is_train)
                print('conv3 layer: ' + str(en_relu3.get_shape()))
                x_unrol = tf.contrib.layers.flatten(en_drop3)
                print('flatten data: ' + str(x_unrol.get_shape()))
                full_1 = x_unrol
                
                #full_1 = tf.layers.dense(x_unrol, units=2048, activation=leaky_relu)
                #print("full_1  : ",str(full_1.get_shape()))
                
                mean = tf.layers.dense(full_1, units=self.n_latent)
                std_val = 0.5 * tf.layers.dense(full_1, units=self.n_latent)   
                epsilon = tf.random_normal(tf.stack([tf.shape(full_1)[0], self.n_latent])) 
                z  = mean + tf.multiply(epsilon, tf.exp(std_val))
                print("latent dimension mean: ",str(mean.get_shape()))
                print("latent dimension std: ",str(std_val.get_shape()))
                print("latent dimension z  : ",str(z.get_shape()))
                
                return z,mean,std_val
    
    def decoder(self,sampled_z):
        dec_in_channels = 1
        reshaped_dim = [-1, 8, 8, 128]
        print("decoder z  : ",str(sampled_z.get_shape()))
        with tf.variable_scope("decoder", reuse=None):
                dense_1 = tf.layers.dense(sampled_z, units=64, activation=leaky_relu)
                print("x_1  : ",str(dense_1.get_shape()))
                dense_2 = tf.layers.dense(dense_1, units=8*8*128, activation=leaky_relu)
                print("x_2  : ",str(dense_2.get_shape()))
                dense_3 = tf.reshape(dense_2, reshaped_dim)
                print("input  : ",str(dense_3.get_shape()))
                conv1 = tf.layers.conv2d_transpose(dense_3, filters=64, kernel_size=4, 
                                    strides=2, padding='same', activation=leaky_relu)
                de_conv1 = dropout(conv1,self.keep_prob,self.is_train)
                print("x_4  : ",str(conv1.get_shape()))
                conv2 = tf.layers.conv2d_transpose(de_conv1, filters=32, kernel_size=4,
                                    strides=2,padding='same', activation=leaky_relu)
                de_conv2 = dropout(conv2,self.keep_prob,self.is_train)
                print("x_5  : ",str(conv2.get_shape()))
                img = tf.layers.conv2d_transpose(de_conv2, filters=3, kernel_size=4,
                                    strides=2,padding='same', activation=tf.sigmoid)
                print("img  : ",str(img.get_shape()))
                return img
            
    def _build_model(self):
        self.z,self.mean,self.std_val=self.encoder(self.X)
        self.d_image = self.decoder(self.z)
        unreshaped = tf.reshape(self.d_image, [-1, self.input_w*self.input_w*3])
        print("unreshaped  : ",str(unreshaped.get_shape()))
        #self.img_loss = tf.reduce_sum(tf.squared_difference(unreshaped, self.X_flat), 1)
        self.recon_loss = -tf.reduce_sum( self.X_flat * tf.log(1e-10+unreshaped) +
                                       (1-self.X_flat) * tf.log(1e-10+(1-unreshaped)),
                                      axis = 1)
        self.latent_loss = -0.5 * tf.reduce_sum(1.0 + 2.0 * self.std_val 
                                           - tf.square(self.mean) 
                                           - tf.exp(2.0 * self.std_val), axis = 1)
        self.total_loss = tf.reduce_mean(self.recon_loss + self.latent_loss)
        self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.total_loss)
        return
    
    def train(self,sess,X_train):
        
        tot_loss = []
        tot_reconloss = []
        tot_latentoss = []
        
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(self.num_epoch):
            print('train for epoch %d' % epoch)
            for i in range(num_training // self.batch_size):
                X_ = X_train[i * self.batch_size:(i + 1) * self.batch_size][:]
                feed_dict = {self.X : X_,self.is_train : True}
                fetches = [self.train_op, self.total_loss, self.recon_loss, self.latent_loss]
                
                _, loss, recon_loss, latent_loss = sess.run(fetches, feed_dict=feed_dict)
                tot_loss.append(loss)
                tot_reconloss.append(np.mean(recon_loss))
                tot_latentoss.append(np.mean(latent_loss))
                
            if epoch % 1 == 0:
                 print('[Epoch {}] Loss: {}, Recon loss: {}, Latent loss: {}'.format(
                            epoch, loss, np.mean(recon_loss), np.mean(latent_loss)))
        
        #plot losses
        plt.subplot(212)
        plt.title('Recon loss')
        recon_hist_ = tot_reconloss[1::50]
        plt.plot(recon_hist_, '-o')
        plt.subplot(221)
        plt.title('Latent loss')
        latent_hist_ = tot_latentoss[1::50]
        plt.plot(latent_hist_, '-o')
        plt.subplot(222)
        loss_hist = tot_loss[1::50]
        plt.plot(loss_hist, '-o')
        plt.show()
        return loss_hist,recon_hist_,latent_hist_
    
    # generate an image from the decoder: z->x
    def generate_sample(self,sess,z):
        gen_image = sess.run(self.d_image, feed_dict={self.z: z,self.is_train : False})
        return gen_image
    
    # reconstruct an input image using encoder-decoder: X->d_image
    def reconstruct_sample(self,sess,X_input):
        gen_image = sess.run(self.d_image, feed_dict={self.X : X_input,self.is_train : False})
        return gen_image
    
    # transform input image into the latent representation: X->z
    def transfor_input(self,sess,X_input):
        fetches = [self.z,self.mean,self.std_val]
        z_val,mean,std_val = sess.run(fetches,feed_dict={self.X : X_input,self.is_train : False})
        return z_val,mean,std_val 

In [ ]:
# Clear old computation graphs
suffel_data=np.random.permutation(train_data.elements)
X_train = train_data.feature_vec[suffel_data][:]
Y_train = train_data.labels[suffel_data]
for val in range(0,5):
    suffel_data=np.random.permutation(train_data.elements)
    X_temp = X_train[suffel_data][:]
    Y_temp = Y_train[suffel_data]
    X_train = X_temp
    Y_train = Y_temp 
    
tf.reset_default_graph()

sess = tf.Session()
model = VAE()
loss_hist,recon_hist_,latent_hist_=model.train(sess,X_train)
print("------done------")

In [ ]:
z = np.random.normal(size=[model.batch_size, model.n_latent])
#z = np.ones((model.batch_size, model.n_latent))
x_generated = model.generate_sample(sess,z)
print(x_generated.shape)

In [ ]:
viz_val=10
print(z[viz_val])
fig, (ax0, ax1,ax2) = plt.subplots(ncols=3,figsize=(16, 8))
ax0.imshow(x_generated[viz_val,:,:,0],cmap=plt.cm.cool)
ax1.imshow(x_generated[viz_val,:,:,1],cmap=plt.cm.cool)
ax2.imshow(x_generated[viz_val,:,:,2],cmap=plt.cm.cool)
plt.show()
#fig, ax0 = plt.subplots(ncols=1,figsize=(16, 8))
tt=None
tt=(x_generated[viz_val,:,:,0]/2.0)+(x_generated[viz_val,:,:,1]*1.0)+(x_generated[viz_val,:,:,2]/3.0)
plt.imshow(tt,cmap=plt.cm.cool)
plt.show()

In [ ]:
x_reconstruct = model.reconstruct_sample(sess,X_train[0:model.batch_size][:])
print(x_reconstruct.shape)

In [ ]:
viz_val=5
fig, (ax0, ax1,ax2) = plt.subplots(ncols=3,figsize=(16, 8))
ax0.imshow(X_train[viz_val,:,:,0],cmap=plt.cm.cool)
ax1.imshow(X_train[viz_val,:,:,1],cmap=plt.cm.cool)
ax2.imshow(X_train[viz_val,:,:,2],cmap=plt.cm.cool)
plt.show()
fig, (ax0, ax1,ax2) = plt.subplots(ncols=3,figsize=(16, 8))
ax0.imshow(x_reconstruct[viz_val,:,:,0],cmap=plt.cm.cool)
ax1.imshow(x_reconstruct[viz_val,:,:,1],cmap=plt.cm.cool)
ax2.imshow(x_reconstruct[viz_val,:,:,2],cmap=plt.cm.cool)
plt.show()

In [ ]:
#visulize latent space
tot_eleval = int(num_training /  model.batch_size)
z_val=np.empty((tot_eleval*model.batch_size,model.n_latent),dtype=float)
print(tot_eleval,tot_eleval*model.batch_size,z_val.shape)
for i in range(num_training // model.batch_size):
    X_ = X_train[i * model.batch_size:(i + 1) * model.batch_size][:]
    aa,_,_=model.transfor_input(sess,X_)
    z_val[i * model.batch_size:(i + 1) * model.batch_size]=aa
print(z_val.shape,len(z_val))

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
pca = PCA(n_components=2,whiten=False)
pca.fit(z_val)
X=pca.transform(z_val)
print("PCA variance ratio:",pca.explained_variance_ratio_)
plt.scatter(X[:,0],X[:,1],c=Y_train[0:len(z_val)])
plt.colorbar(ticks=np.arange(np.min(Y_train),np.max(Y_train)+1))
plt.show()

tsne_model = TSNE(n_components=2,perplexity=50.0,learning_rate=200.0,init='random',verbose = 0)  #verbose 1 is good
X_embedded = tsne_model.fit_transform(z_val)
print("shape of transfored features",X.shape,X_embedded.shape)
plt.scatter(X_embedded[:,0],X_embedded[:,1],c=Y_train[0:len(z_val)])
plt.colorbar(ticks=np.arange(np.min(Y_train),np.max(Y_train)+1))
plt.show()
print("t-SNE Output: KL-Divergence:",tsne_model.kl_divergence_,"Steps: ",tsne_model.n_iter_)

In [ ]:
viz_val=25
bsize = model.batch_size
i_2H=np.argwhere(train_data.labels==0)
nsample_2H=int(len(i_2H)/bsize)
x_2Hreconstruct=np.empty((nsample_2H*bsize,64,64,3),dtype=float)
viz_2H=i_2H[0:nsample_2H*bsize,0]
input_2H=train_data.feature_vec[viz_2H][:]
print(input_2H.shape)
for i in range(nsample_2H):
    X_ = input_2H[i * model.batch_size:(i + 1) * model.batch_size][:]
    aa=model.reconstruct_sample(sess,X_)
    x_2Hreconstruct[i * model.batch_size:(i + 1) * model.batch_size]=aa
fig, (ax0, ax1,ax2) = plt.subplots(ncols=3,figsize=(16, 8))
ax0.imshow(x_2Hreconstruct[viz_val,:,:,0],cmap=plt.cm.cool)
ax1.imshow(x_2Hreconstruct[viz_val,:,:,1],cmap=plt.cm.cool)
ax2.imshow(x_2Hreconstruct[viz_val,:,:,2],cmap=plt.cm.cool)
plt.show()
#-----------------------1T-------------
i_1T=np.argwhere(train_data.labels==1)
nsample_1T=int(len(i_1T)/bsize)
x_1Treconstruct=np.empty((nsample_1T*bsize,64,64,3),dtype=float)
viz_1T=i_1T[0:nsample_1T*bsize,0]
input_1T=train_data.feature_vec[viz_1T][:]
for i in range(nsample_1T):
    X_ = input_1T[i * model.batch_size:(i + 1) * model.batch_size][:]
    aa=model.reconstruct_sample(sess,X_)
    x_1Treconstruct[i * model.batch_size:(i + 1) * model.batch_size]=aa
fig, (ax0, ax1,ax2) = plt.subplots(ncols=3,figsize=(16, 8))
ax0.imshow(x_1Treconstruct[viz_val,:,:,0],cmap=plt.cm.cool)
ax1.imshow(x_1Treconstruct[viz_val,:,:,1],cmap=plt.cm.cool)
ax2.imshow(x_1Treconstruct[viz_val,:,:,2],cmap=plt.cm.cool)
plt.show()
#----------------------defect---------------
i_defect=np.argwhere(train_data.labels==2)
nsample_d=int(len(i_defect)/bsize)
x_dreconstruct=np.empty((nsample_d*bsize,64,64,3),dtype=float)
viz_de=i_defect[0:nsample_d*bsize,0]
input_de=train_data.feature_vec[viz_de][:]
for i in range(nsample_d):
    X_ = input_de[i * model.batch_size:(i + 1) * model.batch_size][:]
    aa=model.reconstruct_sample(sess,X_)
    x_dreconstruct[i * model.batch_size:(i + 1) * model.batch_size]=aa
fig, (ax0, ax1,ax2) = plt.subplots(ncols=3,figsize=(16, 8))
ax0.imshow(x_dreconstruct[viz_val,:,:,0],cmap=plt.cm.cool)
ax1.imshow(x_dreconstruct[viz_val,:,:,1],cmap=plt.cm.cool)
ax2.imshow(x_dreconstruct[viz_val,:,:,2],cmap=plt.cm.cool)
plt.show()
print("total 2H:",len(i_2H),nsample_2H)
print("total 1T:",len(i_1T),nsample_1T)
print("total defect:",len(i_defect),nsample_d)

In [ ]:
zval_2H=np.empty((nsample_2H*bsize,model.n_latent),dtype=float)
mu_2H=np.empty((nsample_2H*bsize,model.n_latent),dtype=float)

zval_1T=np.empty((nsample_1T*bsize,model.n_latent),dtype=float)
mu_1T=np.empty((nsample_1T*bsize,model.n_latent),dtype=float)

zval_de=np.empty((nsample_d*bsize,model.n_latent),dtype=float)
mu_de=np.empty((nsample_d*bsize,model.n_latent),dtype=float)

for i in range(nsample_2H):
    X_ = input_2H[i * model.batch_size:(i + 1) * model.batch_size][:]
    aa,bb,_ = model.transfor_input(sess,X_)
    zval_2H[i * model.batch_size:(i + 1) * model.batch_size]=aa
    mu_2H[i * model.batch_size:(i + 1) * model.batch_size]=bb

for i in range(nsample_1T):
    X_ = input_1T[i * model.batch_size:(i + 1) * model.batch_size][:]
    aa,bb,_ = model.transfor_input(sess,X_)
    zval_1T[i * model.batch_size:(i + 1) * model.batch_size]=aa
    mu_1T[i * model.batch_size:(i + 1) * model.batch_size]=bb

for i in range(nsample_d):
    X_ = input_de[i * model.batch_size:(i + 1) * model.batch_size][:]
    aa,bb,_ = model.transfor_input(sess,X_)
    zval_de[i * model.batch_size:(i + 1) * model.batch_size]=aa
    mu_de[i * model.batch_size:(i + 1) * model.batch_size]=bb
    
mu_2H_avg=np.tile(np.average(mu_2H,axis=0),(64,1))
mu_1T_avg=np.tile(np.average(mu_1T,axis=0),(64,1))
mu_de_avg=np.tile(np.average(mu_de,axis=0),(64,1))
print("shape Z:",zval_2H.shape,zval_1T.shape,zval_de.shape)
print("shape mu:",mu_2H_avg.shape,mu_1T_avg.shape,mu_de_avg.shape)
print("==done===")

In [ ]:
delta=0
viz_val=50
diff_z=mu_2H_avg-mu_1T_avg

#angle_t=np.arccos(np.dot(zval_2H[viz_val,:],zval_1T[viz_val,:]))

#diff_z=mu_2H-mu_1T
x_new = model.generate_sample(sess,delta*zval_2H[0*64:1*64][:]+(1-delta)*zval_1T[0*64:1*64][:])
#x_new = model.generate_sample(sess,zval_2H[1*64:2*64][:]-delta*(mu_2H_avg-mu_1T_avg))
#x_new = model.generate_sample(sess,zval_2H[0:64][:]-delta*(mu_2H[0:64][:]-mu_1T[0:64][:]))
#x_new = model.generate_sample(sess,-0.3*zval_1T[0:64][:])

#ax=(np.sin((1-delta)*angle_t))/np.sin(angle_t)
#ay=(np.sin(delta*angle_t))/np.sin(angle_t)
#print("ax and ay:",ax,ay)
#x_new = model.generate_sample(sess,ax*zval_2H[0:64][:]+ay*zval_1T[0:64][:])

print("===done===")
print(diff_z[viz_val])
fig, (ax0, ax1,ax2,ax3) = plt.subplots(ncols=4,figsize=(16, 8))
ax0.imshow(x_new[viz_val,:,:,0],cmap=plt.cm.binary)
ax1.imshow(x_new[viz_val,:,:,2],cmap=plt.cm.binary)
ax2.imshow(x_new[viz_val,:,:,1],cmap=plt.cm.binary)
tt=None
tt=(x_new[viz_val,:,:,0]/2.0)+(x_new[viz_val,:,:,1]*1.0)+(x_new[viz_val,:,:,2]/2.0)
ax3.imshow(tt,cmap=plt.cm.binary)
plt.show()

In [ ]:
Se1=x_new[viz_val,:,:,0]
Se2=x_new[viz_val,:,:,2]
Mo=x_new[viz_val,:,:,1]
dim=Se1.shape[0]
print("dimension of tensor:",dim)
atom_tensor=[Se1,Se2,Mo]
zval=[-1.65,1.65,0.0]
atype=['Se1','Se2','Mo']
print(atom_tensor[0].shape)

In [ ]:
#make neighborlist
delta = 20.0/64.0
thresh=0.1
for ii in range(0,dim):
    for jj in range(0,dim):
        myid=dim*ii+jj
        count=0
        for kk in range(ii-1,ii+2):
            for ll in range(jj-1,jj+2):
                if kk==ii and ll==jj:
                    continue
                elif kk < 0 or kk >= dim or ll < 0 or ll >= dim:
                    continue
                else:
                    count +=1
                    child_id=dim*kk+ll
                    nlist[myid][count]=child_id
        nlist[myid][0]=count
        #print("nlist",nlist[myid,:])
        #print(ii,jj,":",myid,":",i_val,j_val,nlist[myid][0],count) 


def dfs_search(node,atom_id,layer_tag,nlist,nflag):
    nflag[node]=atom_id
#    print("hi",node,nlist[node,0:nlist[node,0]+1])
    for ii in range(1,nlist[node][0]+1):
        child_id = nlist[node][ii]
        i_ch = child_id // dim
        j_ch = child_id % dim
        if layer_tag[i_ch][j_ch] > thresh and nflag[child_id] == 0 :
            dfs_search(child_id,atom_id,layer_tag,nlist,nflag)
    return

def gen_coordinate(layer_tag,nlist,nflag):
    count=0
    for loc in range(0,dim*dim):
        i_val=loc // dim
        j_val=loc % dim
        if layer_tag[i_val][j_val]> thresh and nflag[loc] == 0 :
            count+=1
            dfs_search(loc,count,layer_tag,nlist,nflag)
    return count

In [ ]:
layer_num=-1
total_atom=0
for val in atom_tensor:
    layer_num+=1
    nlist=np.empty((dim*dim,9),dtype=int)
    nflag=np.zeros(dim*dim,dtype=int)
    count=gen_coordinate(val,nlist,nflag)
    total_atom+=count
    x_cor=np.zeros(count+1,dtype=float)
    y_cor=np.zeros(count+1,dtype=float)
    tot=np.zeros(count+1,dtype=float)
    for ii in range(0,dim*dim):
        if nflag[ii] != 0:
            i_val=ii // dim
            j_val=ii % dim
            tot[nflag[ii]] += 1.0
            x_cor[nflag[ii]] += i_val*delta
            y_cor[nflag[ii]] += j_val*delta
    for ii in range(1,count+1):
        x_cor[ii]=x_cor[ii]/tot[ii]
        y_cor[ii]=y_cor[ii]/tot[ii]
        print(atype[layer_num],x_cor[ii],y_cor[ii],zval[layer_num])
print("total_atom: ",total_atom)

In [ ]:
val='Spectral'
plt.scatter(X[:,0],X[:,1],c=Y_train[0:len(z_val)],cmap=val)
plt.colorbar(ticks=np.arange(np.min(Y_train),np.max(Y_train)+1))
plt.savefig("pca.png",dpi=300)
plt.show()

plt.scatter(X_embedded[:,0],X_embedded[:,1],c=Y_train[0:len(z_val)],cmap=val)
plt.colorbar(ticks=np.arange(np.min(Y_train),np.max(Y_train)+1))
plt.savefig("tsne.png",dpi=300)
plt.show()

In [ ]:
print(train_data2.labels.shape)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
pca = PCA(n_components=3,whiten=False)
pca.fit(z_val)
X=pca.transform(z_val)
print("PCA variance ratio:",pca.explained_variance_ratio_)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
p=ax.scatter(X[:,0],X[:,2],X[:,1],c=Y_train[0:len(z_val)],cmap=val)
fig.colorbar(p)
fig.savefig("3dpca.png",dpi=300)
plt.show()

In [ ]:
tsne_model = TSNE(n_components=2,perplexity=30.0,learning_rate=200.0,init='random',verbose = 0)  #verbose 1 is good
X_embedded = tsne_model.fit_transform(z_val)
print("shape of transfored features",X.shape,X_embedded.shape)
plt.scatter(X_embedded[:,0],X_embedded[:,1],c=Y_train[0:len(z_val)])
plt.colorbar(ticks=np.arange(np.min(Y_train),np.max(Y_train)+1))
plt.show()
print("t-SNE Output: KL-Divergence:",tsne_model.kl_divergence_,"Steps: ",tsne_model.n_iter_)

In [ ]:
plt.scatter(X_embedded[:,0],X_embedded[:,1],c=Y_train[0:len(z_val)],cmap=val)
plt.colorbar(ticks=np.arange(np.min(Y_train),np.max(Y_train)+1))
plt.savefig("tsne.png",dpi=300)
plt.show()

In [ ]:
np.arccos(np.dot(zval_2H[50,:],zval_1T[50,:]))*180/3.14